In [236]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil


def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv'):
            os.remove(os.path.join(download_path, file))
    
    combined_df.to_csv(f"url_data.csv", index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df



def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

    return len(failed_downloads) == 0  # 모든 다운로드가 성공했으면 True 반환



def compress_images_to_zip(download_path, post_href, post_user):
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    duplicate_url = combine_csv_files_remove_duplicates('.')
                      
    failed_posts = []
    i = 1
    total = len(post_list)
    success_url = []
    old_urls = duplicate_url.loc[:, 'url'].to_list()
    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            if not href in old_urls : 
                print(f"Processing: {total} 중 {i}  {title}  _  {user}")
                success = download_images_from_links(driver, href, download_path, title)
                is_successful_zip = compress_images_to_zip(download_path, href, user)
                if not is_successful_zip or not success:
                    failed_posts.append((href,title,user))
                    print(f"failed : {href}")
                else :
                    success_url.append([href, title]) 
            else:
                print(f"중복 : {title}")
        i=i+1

    print("_____finished______")
    print("Failed posts:")
    for url, title, user in failed_posts:
        print(f"User: {user} Title: {title}, URL: {url}")
                      
    if success_url:
        temp = pd.DataFrame(success_url)
        temp.columns = ['url', 'filename']
        temp.to_csv(f"Arca_url_data.csv", index=False)
    else:
        print("No data available to create DataFrame")
    
    return failed_posts



    


In [237]:

download_path = '/Users/jonathanoh/Desktop/down'
excel_path = '/Users/jonathanoh/Desktop/down/image_url.xlsx'

options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')

In [216]:


# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
post_list


#process_images(driver, post_list, download_path)

[('https://arca.live/b/aiartreal/94895726?p=1',
  '산타걸 TPix_typeRE_v1.6_fp16 찍먹 냠냠',
  '현무'),
 ('https://arca.live/b/aiartreal/94895265?p=1', 'Ai야 가수들도 다 죽일셈이냐?', '리버스Q'),
 ('https://arca.live/b/aiartreal/94894908?p=1', '외계인 침공', 'igo'),
 ('https://arca.live/b/aiartreal/94894548?p=1', 'Merry Chirstmas', 'leezi'),
 ('https://arca.live/b/aiartreal/94894255?p=1',
  '오랜만에 시간이 나서 몇장 올려봅니다ㅎ',
  '냐오미잉'),
 ('https://arca.live/b/aiartreal/94892602?p=1',
  '꿈속에서 만나실 산타걸^^...(스압)',
  '실사좋아'),
 ('https://arca.live/b/aiartreal/94891333?p=1',
  '마이너장르 -피규어 건전짤 투척(DENG_e9.fp16 모델)',
  '냥이집사'),
 ('https://arca.live/b/aiartreal/94889737?p=1',
  'Comfyui SDXL 그림 테스트 - 사이버펑크 & 크리스마스',
  'thec'),
 ('https://arca.live/b/aiartreal/94889717?p=1',
  'hanimix_real_v15A 쩍먹해보았습니다.',
  '삼선짬뽕'),
 ('https://arca.live/b/aiartreal/94889053?p=1',
  'One Bar Prison (하드..?)',
  '아리아재'),
 ('https://arca.live/b/aiartreal/94888024?p=1',
  'hanimix_real_v15A 찍먹',
  '리스펙트'),
 ('https://arca.live/b/aiartreal/94888011?p=1', '베

In [238]:
post_list = []
for i in range(2):
    driver.get(f"https://arca.live/b/aiartreal?p={i+1}")
    if i == 0:
        post_list = post_list+fetch_post_list(driver)[7:]
    else :
        post_list = post_list+fetch_post_list(driver)
post_list    

[('https://arca.live/b/aiartreal/94964439?p=1', '이게 뭐야 !!!!', 'dk카라브'),
 ('https://arca.live/b/aiartreal/94963557?p=1', 'NeMix VurfY / 제한 공유', '북이'),
 ('https://arca.live/b/aiartreal/94960434?p=1',
  'NeMix Real-D v1.4.0 수영복 찍먹',
  'carmen'),
 ('https://arca.live/b/aiartreal/94959491?p=1', '이것저것', 'OTL'),
 ('https://arca.live/b/aiartreal/94958822?p=1',
  '거미줄 무늬 옷 (NeMix F v2.8.0)',
  '너예쁘다'),
 ('https://arca.live/b/aiartreal/94958743?p=1', 'BMAB 이미지 합성이 안되는데', '젬마'),
 ('https://arca.live/b/aiartreal/94958197?p=1',
  '뉴비가 올려보는 크리스마스 시즌기념 시스루 눈나!',
  'Ai우유'),
 ('https://arca.live/b/aiartreal/94958109?p=1',
  '소재) 식빵 출근 - 서술형 프롬 / NeMix F v2.8.0 찍먹',
  'Gbox'),
 ('https://arca.live/b/aiartreal/94957656?p=1', '그시절 우리가 사랑했던 소녀', '빠다파리'),
 ('https://arca.live/b/aiartreal/94957608?p=1', '골방녀 병맛판', '고삐풀린뉴비'),
 ('https://arca.live/b/aiartreal/94957423?p=1', '사이보그', '빠다파리'),
 ('https://arca.live/b/aiartreal/94957268?p=1', '어라라.. 선물이 잘못온것같다.', 'dk카라브'),
 ('https://arca.live/b/aiartreal/94957234?

In [240]:
post_list = post_list[:-15]

In [241]:
list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)

Processing: 75 중 1  이게 뭐야 !!!!  _  dk카라브
Processing: 75 중 2  NeMix VurfY / 제한 공유  _  북이
다운로드 중 오류 발생: Message: no such element: No node with given id found
  (Session info: chrome=120.0.6099.109)
Stacktrace:
0   chromedriver                        0x0000000102ffc4cc chromedriver + 4162764
1   chromedriver                        0x0000000102ff4654 chromedriver + 4130388
2   chromedriver                        0x0000000102c4bbc0 chromedriver + 293824
3   chromedriver                        0x0000000102c340d8 chromedriver + 196824
4   chromedriver                        0x0000000102c31e84 chromedriver + 188036
5   chromedriver                        0x0000000102c32b30 chromedriver + 191280
6   chromedriver                        0x0000000102c5a5c4 chromedriver + 353732
7   chromedriver                        0x0000000102c51af0 chromedriver + 318192
8   chromedriver                        0x0000000102c51bc8 chromedriver + 318408
9   chromedriver                        0x0000000102c5017c ch

Processing: 75 중 27  chronomancer  _  바코드
Processing: 75 중 28  터틀넥  _  빠다파리
Processing: 75 중 29  (nude:1.4), 받고 더블로간 멋드러진 짤  _  머리는차갑게가슴은뜨겁게
Processing: 75 중 30  스타포스  _  빠다파리
Processing: 75 중 31  slingshot  _  바코드
Processing: 75 중 32  타임머신  _  빠다파리
failed : https://arca.live/b/aiartreal/94942935?p=1
Processing: 75 중 33  1장) 슬링샷 비키니  _  그림꾼달잡이
Processing: 75 중 34  (nude:1.4), 를 뚫고나온 멋드러진 짤  _  머리는차갑게가슴은뜨겁게
Processing: 75 중 35  포토샵 - 리무브 펜스(픽셀선택방법)  _  냥이집사
다운로드 중 오류 발생: Message: no such element: No node with given id found
  (Session info: chrome=120.0.6099.109)
Stacktrace:
0   chromedriver                        0x0000000102ffc4cc chromedriver + 4162764
1   chromedriver                        0x0000000102ff4654 chromedriver + 4130388
2   chromedriver                        0x0000000102c4bbc0 chromedriver + 293824
3   chromedriver                        0x0000000102c340d8 chromedriver + 196824
4   chromedriver                        0x0000000102c31e84 chromedriver + 188036
5   chromedr

In [247]:
list = process_images(driver, list, download_path)

Processing: 11 중 1  NeMix VurfY / 제한 공유  _  북이
다운로드 중 오류 발생: Message: no such element: No node with given id found
  (Session info: chrome=120.0.6099.109)
Stacktrace:
0   chromedriver                        0x0000000102ffc4cc chromedriver + 4162764
1   chromedriver                        0x0000000102ff4654 chromedriver + 4130388
2   chromedriver                        0x0000000102c4bbc0 chromedriver + 293824
3   chromedriver                        0x0000000102c340d8 chromedriver + 196824
4   chromedriver                        0x0000000102c31e84 chromedriver + 188036
5   chromedriver                        0x0000000102c32b30 chromedriver + 191280
6   chromedriver                        0x0000000102c5a5c4 chromedriver + 353732
7   chromedriver                        0x0000000102c51af0 chromedriver + 318192
8   chromedriver                        0x0000000102c51bc8 chromedriver + 318408
9   chromedriver                        0x0000000102c5017c chromedriver + 311676
10  chromedriver     

In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [234]:
temp = []
for url, title, user in list :
    temp.append([url, title])
df = pd.DataFrame(temp)
df.columns=['url', 'filename']
df.to_csv("fail_list.csv", index=False)
df = combine_csv_files_remove_duplicates('.')